## Using MLOps DRUM to test your custom models
**Author**: Matthew Cohen

#### Scope
The scope of this Notebook is to provide examples for using the MLOps DRUM library to test your custom model locally. 

This includes an example for both a regression and binary classification example:
1. Create a new random foreset models
1. Implement a function in custom.py to do additional prediction request pre/post processing
1. Validate they stand up to errors in input data
1. Request predictions with a test dataset

There are also examples to: 
- Test batch predictions
- Run drum as a web service
- Train a custom model

#### Requirements

Start by following the Quickstart instruction on https://github.com/datarobot/datarobot-user-models to set up your environment and perform the necessary installations.   

Additional information can be found here:  
- https://github.com/datarobot/datarobot-user-models/tree/master/custom_model_runner
- https://pypi.org/project/datarobot-drum/



In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

## 1) Train a regression model

A simple RandomForestRegressor to predict house prices in Boston.

In [4]:
# Read the train and test data
TRAIN_DATA_REG = './data/boston_housing_train.csv'  # 14 features
TEST_DATA_REG = './data/boston_housing_test.csv'  # 13 features - target is removed

reg_X_train = pd.read_csv(TRAIN_DATA_REG)
reg_Y_train = reg_X_train.pop('MEDV')

reg_X_test = pd.read_csv(TEST_DATA_REG)

# Fit the model
reg_rf_model = RandomForestRegressor()
reg_rf_model.fit(reg_X_train, reg_Y_train)

# Pickle the file and write it to the file system
if not os.path.exists('custom_model_reg'):
    os.makedirs('custom_model_reg')
with open('custom_model_reg/reg_rf_model.pkl', 'wb') as pkl:
    pickle.dump(reg_rf_model, pkl)
    
# Call predict to confirm it works
reg_rf_model.predict(reg_X_test)

array([25.999, 21.929, 34.349, 33.893, 35.548, 27.352, 22.31 , 22.816,
       16.792, 18.965, 16.657, 19.785, 21.483, 20.262, 18.74 , 20.086,
       22.764, 17.737, 19.668, 18.989])

## 2) Generate the model template file for any additional pipeline processing

This file, custom.py, is optional but allows you to insert additional processing steps into the flow of getting predictions.  The following functions are available:

* init
* load_model
* transform
* score
* post_process

Place the file in the location specified by the --code-dir argument.  For this example, you must edit the transform function in custom.py to impute any null values to 0.  Please see the comments in custom.py for further description information of each function.

In [5]:
# Create a new directory with a custom.py template
!drum new model --code-dir ./custom_model_reg/ --language python

usage: drum new model [-h] [--verbose] -cd CODE_DIR
                      [--logging-level {noset,debug,info,warn,warning,error,critical}]
                      --language {python,r}
drum new model: error: argument -cd/--code-dir: The path ./custom_model_reg/ already exists! Please provide a non existing path!


## 3) Validate the regression model can handle data with errors

The validation check takes the input file and alters it to test various fail conditions, such as setting column values to null.  For this example, you must edit the transform function in custom.py to impute any null values to 0.

In [6]:
!drum validation --code-dir ./custom_model_reg --input data/boston_housing_test.csv

    Predictions
0        25.999
1        21.929
2        34.244
3        33.789
4        34.846
5        27.297
6        21.853
7        22.668
8        18.798
9        19.471
10       19.653
11       19.688
12       21.449
13       19.963
14       18.742
15       19.874
16       22.405
17       18.525
18       19.510
19       19.015
    Predictions
0        25.983
1        21.929
2        34.349
3        33.893
4        35.548
5        27.352
6        22.271
7        22.645
8        16.773
9        18.938
10       16.669
11       19.762
12       21.491
13       20.262
14       18.740
15       20.086
16       22.764
17       17.737
18       19.668
19       18.989
    Predictions
0        26.036
1        22.108
2        34.662
3        33.947
4        35.586
5        27.310
6        22.848
7        22.827
8        16.808
9        19.009
10       16.737
11       20.662
12       21.488
13       20.585
14       19.795
15       20.286
16       22.502
17       18.207
18       20.546
19      

## 4) Test the regression model can return predictions 

Input the prediction dataset that includes all features except the target feature.

In [7]:
!drum score --code-dir ./custom_model_reg/ --input data/boston_housing_test.csv --output cmrunner_test_pred_results.csv --verbose

Detected score mode
Start initializing pipeline
Detected /Users/matthew.cohen/Documents/DR/MLOps/_DRUM local testing/DRUM notebook example/custom_model_reg/custom.py .. trying to load hooks
Start running pipeline
 
 
Component: csv_to_df
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 1
 
 
 
Component: python_predictor
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 1
 
 
 
Component: df_to_csv
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 0
 
Finish running pipeline


## 1) Train a binary classification model

This flow is essentially the same as a regression model but uses slightly different arguments for drum.

In [8]:
# Read the train and test data
TRAIN_DATA_CLF = './data/surgical_dataset_train.csv'
TEST_DATA_CLF = './data/surgical_dataset_test.csv'

clf_X_train = pd.read_csv(TRAIN_DATA_CLF)
clf_Y_train = clf_X_train.pop('complication')

clf_X_test = pd.read_csv(TEST_DATA_CLF)

# Fit the model
clf_rf_model = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
clf_rf_model.fit(clf_X_train, clf_Y_train)

# Pickle the file and write it to the file system
if not os.path.exists('custom_model_clf'):
    os.makedirs('custom_model_clf')
with open('custom_model_clf/clf_rf_model.pkl', 'wb') as pkl:
    pickle.dump(clf_rf_model, pkl)
    
# Call predict to confirm it works
clf_rf_model.predict(clf_X_test)

threshold = 0.3
predicted_proba = clf_rf_model.predict_proba(clf_X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')
predicted
# accuracy_score(clf_Y_test, predicted)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1])

## 2) Generate the model template file for any additional pipeline processing

This file, custom.py, is optional but allows you to insert additional processing steps into the flow of getting predictions.  The following functions are available:

* init
* load_model
* transform
* score
* post_process

Place the file in the location specified by the --code-dir argument.  For this example, you must edit the transform function in custom.py to impute any null values to 0.  Please see the comments in custom.py for further description information of each function.

In [9]:
# Create a new directory with a custom.py template
!drum new model --code-dir ./custom_model_clf/ --language python

usage: drum new model [-h] [--verbose] -cd CODE_DIR
                      [--logging-level {noset,debug,info,warn,warning,error,critical}]
                      --language {python,r}
drum new model: error: argument -cd/--code-dir: The path ./custom_model_clf/ already exists! Please provide a non existing path!


## 3) Validate the classification model can handle data with errors

The validation check takes the input file and alters it to test various fail conditions, such as setting column values to null.  For this example, you must edit the transform function in custom.py to impute any null values to 0.

In [10]:
!drum validation --code-dir ./custom_model_clf --input data/surgical_dataset_test.csv  --positive-class-label 1 --negative-class-label 0

           1         0
0   0.043216  0.956784
1   0.264750  0.735250
2   0.274012  0.725988
3   0.173259  0.826741
4   0.278859  0.721141
5   0.286626  0.713374
6   0.338873  0.661127
7   0.278859  0.721141
8   0.278859  0.721141
9   0.281780  0.718220
10  0.043216  0.956784
11  0.264545  0.735455
12  0.043216  0.956784
13  0.256982  0.743018
14  0.243151  0.756849
15  0.415967  0.584033
16  0.286626  0.713374
17  0.043216  0.956784
18  0.345199  0.654801
19  0.432839  0.567161
           1         0
0   0.126457  0.873543
1   0.264750  0.735250
2   0.274012  0.725988
3   0.173259  0.826741
4   0.278859  0.721141
5   0.264113  0.735887
6   0.338873  0.661127
7   0.256345  0.743655
8   0.256345  0.743655
9   0.259267  0.740733
10  0.126457  0.873543
11  0.264545  0.735455
12  0.126457  0.873543
13  0.234469  0.765531
14  0.220638  0.779362
15  0.415967  0.584033
16  0.286626  0.713374
17  0.126457  0.873543
18  0.322686  0.677314
19  0.432839  0.567161
           1         0
0   0.04321

^C

Ctrl+C pressed, aborting drum


## 4) Test the classification model can return predictions 

Input the prediction dataset that includes all features except the target feature.

In [11]:
!drum score --code-dir ./custom_model_clf/ --input data/surgical_dataset_test.csv --positive-class-label 1 --negative-class-label 0 --output surgical_complications_test_results.csv --verbose

Detected score mode
Start initializing pipeline
Detected /Users/matthew.cohen/Documents/DR/MLOps/_DRUM local testing/DRUM notebook example/custom_model_clf/custom.py .. trying to load hooks
Start running pipeline
 
 
Component: csv_to_df
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 1
 
 
 
Component: python_predictor
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 1
 
 
 
Component: df_to_csv
Language:  Python
Output:
------------------------------------------------------------
------------------------------------------------------------
Runtime:    0.0 sec
NR outputs: 0
 
Finish running pipeline


## Testing model performance

Use this to asses model response time for prediction requests.

In [12]:
!drum perf-test --code-dir ./custom_model_clf --input data/surgical_dataset_test.csv --positive-class-label 1 --negative-class-label 0

Preparing test data...



Running test case: 96 bytes - 1 samples, 100 iterations
Processing |################################| 100/100
Running test case: 0.1MB - 1091 samples, 50 iterations
Processing |################################| 50/50
Running test case: 10MB - 109113 samples, 5 iterations
Processing |################################| 5/5
Running test case: 50MB - 545566 samples, 1 iterations
Processing |################################| 1/1
>78
  size     samples   iters    min     avg     max    used (MB)   total (MB)
96 bytes         1     100   0.080   0.101   0.171     118.672    16384.000
0.1MB         1091      50   0.133   0.157   0.233     122.770    16384.000
10MB        109113       5   1.092   1.145   1.201     204.301    16384.000
50MB        545566       1   5.685   5.685   5.685     475.270    16384.000


## Prediction server mode

The code below launchs drum as a server and stop program flow.  So to test that it responds to prediction requests, issue this command in a terminal shell or another notebook environment:

curl -F "X=@./data/boston_housing_test.csv" localhost:6789/predict/

In [13]:
!drum server --code-dir ./custom_model_reg --address localhost:6789

^C


## Fit a model

https://github.com/datarobot/datarobot-user-models/blob/master/QUICKSTART-FOR-TRAINING.md

In [14]:
!drum fit --code-dir model_templates/training/python3_sklearn --target complication --input data/surgical_dataset_train.csv --positive-class-label 1 --negative-class-label 0

Validation Complete 🎉 Your model can be fit to your data, and predictions can be made on the fit model! 
You're ready to add it to DataRobot. 
